Group: I-Need-More-Boolets



Members: <br>
Go, Ryan Jefferson <br>
Ejercito, Joshua Carl <br>
Moraña, Anton Louis <br>
Nieva, Samuel



Section: S11



# Introduction to the problem/task and dataset

This major course output tasked our group to select a real-word dataset from the selection given. This group has selected the stars dataset, which is a dataset containing spectral characteristics derived from pictures taken by the SDSS (Sloan Digital Sky Survey). This dataset's main use is spectral classification, i.e., classifying whether a celestial body is a star, a quasar or an entire galaxy.

Using the dataset, the group is to perform the following:

- Describe the dataset
- Perform Data Pre-processing and Cleaning
- Perform Exploratory Data Analysis
- Select 3 Machine Learning Models
- Perform model training
- Perform hyperparameter tuning
- Extract insights from the data



# Description of the dataset

As stated earlier, the Stars datasets contains spectral characteristics from celestial bodies that can be used to classify it as either a quasar, star, or galaxy. This dataset is made up of 100,000 obersavations of space taken by the SDSS (Sloan Digital Sky Survey) with each observations having 18 features. The SDSS takes these observations through the use of a 2.5m wide located at Apache Point Observatory in New Mexico. The telescope is described in detail in a paper by Gunn et al. (2006), and it is through this powerful telescope that a robust astronomical survey is created, from which datasets such as this are derived.

The following are the description of their features:

1. obj_ID =  the unique value that identifies the object in the image catalog used by th CAS (Chinese Academy of Sciences).
2. alpha = it contains the right ascension angle, it is measured from the start of a point which is called vernal equinox and go eastward. It is expressed in hours, minutes, and seconds. Vernal equinox is when the sun is exactly above the equator and day and night are of equal day. It is similar to the longitude in space.
3. delta = it contains the declination angle, which shows the angle between the celestial equator and a point on the celestial sphere. It is like the latitude in space which results can either be positive or negative. Positive means that the object is located north while negative means it is located south.
4. u – Ultraviolet filter in the photometric system.
5. g – Green filter in the photometric system.
6. r – Red filter in the photometric system.
7. i – Near Infrared filter in the photometric system.
8. z – Infrared filter in the photometric system.
9. run_ID – Run Number used to identify the specific scan.
10. rereun_ID – Rerun Number to specify how the image was processed.
11. cam_col – Camera column to identify the scanline within the run.
12. field_ID – Field number to identify each field.
13. spec_obj_ID – Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class).
14. class – object class (galaxy, star or quasar object). This is the label the group will be tasking the models to predict.
15. redshift – redshift value based on the increase in wavelength.
16. plate – plate ID, identifies each plate in SDSS (Sloan Digital Sky Survey).
17. MJD – Modified Julian Date, used to indicate when a given piece of SDSS (Sloan Digital Sky Survey) data was taken.
18. fiber_ID – identifies the fiber that pointed the light at the focal plane in each observation.

The first few rows of the dataset look like this:



In [ ]:
import pandas as pd
import numpy as np

seed_num = 12

np.random.seed(seed_num)

stars = pd.read_csv('stars.csv')
print("stars dataset shape: ", stars.shape)
stars.head()

The following shows the number of data points for each class.

In [ ]:
stars['class'].value_counts()

The following shows the datatypes of each feature within the dataset:

In [ ]:
stars.info()

# List of requirements

The group will be using Python's scikit learn library for their classification models (I think), scipy, scikit, numpy and pandas array for processing data, and matplotlib for visualizing data (add stuff if need pa)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV

# Data Cleaning and Preprocessing



To begin, we assign a set of columns that will be the focus for data cleaning and preprocessing, namely the `u`, `g`, `r`, `i`, `z`, and `redshift` columns.

In [ ]:
columns_to_clean = ['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']

## Replacement of `obj_ID` column



The `obj_ID` column assigns a unique ID for each observation in the dataset, although it is represented in a floating point format. For the purposes of the study, this will be replaced with a similarly-named column that utilizes integer IDs instead in order to simplify the access to each element.

In [ ]:
stars = stars.drop(['obj_ID'], axis=1)
stars['obj_ID'] = np.arange(len(stars))
stars.insert(0, 'obj_ID', stars.pop('obj_ID'))
stars

## Checking for empty observations

Checking was done for any entries with missing/null features by instantiating another stars dataset whose entries are filtered off of null features.

In [ ]:
nan_variables = stars[stars.isnull().any(axis=1)].index
stars_nonull = stars.drop(labels=nan_variables).reset_index(drop=True)
print("stars dataset shape (no null): ", stars_nonull.shape)

As shown by the resulting filtered dataset and comparing its shape to the original dataset, it was shown that all entries have all their features accounted for.

## Checking for duplicate entries

Any duplicated entries was checked first by comparing all entries' columns to each other

In [ ]:
stars.duplicated().any()

As shown, there are no duplicated entries within the dataset.

## Checking for outliers

The removal of outliers aims to craft a dataset whose subset can be used for training data that is generally representative of each classes. We will first check for the existence of outliers in this instance, and any suspicious outliers that implies default/placeholder values will be removed from analysis.


In [ ]:
stars[(np.abs(stats.zscore(stars[columns_to_clean])) < 3).all(axis=1)].shape

There are 2116 data points that are considered as outliers based from the z-score metric (>= 3 standard deviations from the mean). In order to know more about these outliers, the features are drawn in a boxplot, displaying their minimum and maximum values respectively.

In [ ]:
for col in columns_to_clean:
    stars.boxplot(col, figsize=(4, 4))
    plt.tight_layout()
    plt.show()
    print("\"",col,"\" minimum value: ", stars[col].min())
    print("\"",col,"\" maximum value: ", stars[col].max())

Some features have values equated to a whole value of \-9999. This may indicate that these features were not recorded/not available for that observation, thus putting a default value for those features. To resolve this, we will remove the associated entries that has default values for these certain features.


In [ ]:
default_value = stars['u'].min()
stars = stars[stars['u'] != default_value]
stars = stars[stars['g'] != default_value]
stars = stars[stars['z'] != default_value]
stars.shape

Based from the filtered dataset, there is only one entry that contains such default values. Any other outliers will be resolved by applying the robust scaling method for normalization.



In [ ]:
for col in columns_to_clean:
    stars.boxplot(col, figsize=(4, 4))
    plt.tight_layout()
    plt.show()
    print("\"",col,"\" minimum value: ", stars[col].min())
    print("\"",col,"\" maximum value: ", stars[col].max())

The boxplots indicates that any extreme outliers that comes from the default values were successfully removed. However, the `redshift` feature may require additional feature engineering in the form of logarithmic transformation, which will be done in the normalization phase.



## Logarithmic transformation of the `redshift` feature



The `redshift` feature will be transformed by implementing a helper function to be used in the normalization process.

In [ ]:
def log_transform_feature(df, feature):
    feature_min = df[feature].min()
    feature_max = df[feature].max()
    # shifter to convert negaative values to positive, and to keep any values above zero for log computation
    shifter = abs(feature_min) + 0.001
        
    df[feature] = np.log(df[feature] + shifter)
    
    return df

## Fixing `obj_ID` after the removal of outliers

Since the outliers were removed, the row obj_ID may not reflect it's actual position in the dataset. To fix this, we simply reassign the obj_IDs incrementally.



In [ ]:
stars = stars.drop(['obj_ID'], axis=1)
stars['obj_ID'] = np.arange(len(stars))
stars.insert(0, 'obj_ID', stars.pop('obj_ID'))
stars

## Normalization



Normalization is performed for the previously\-specified columns in order to equally weight\-in each of the features for the model training and evaluation. The robust scaler will be applied as normalization technique to account for the massive outliers for some features such as `u`, `g,` and `z`.

In order to begin the normalization process, the original dataset is split into training and test dataset alongside splitting their features and labels, and are then normalized. This was done to simulate the prediction on unseen data by only doing the normalization computations within the split datasets themselves to avoid imposing any bias for the evaluation. Additionally, the redshift feature is log\-transformed first in order to extract more range from its skewed distribution.


In [ ]:
# For use in EDA
og_stars = stars.copy()

cols = ['u','g','r','i','z','redshift']
X = stars[cols]
y = stars["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=seed_num, test_size=0.3, stratify=y)

# log-transform redshift features first
X_train = log_transform_feature(X_train, "redshift")
scaler = RobustScaler().fit(X_train[cols])
X_train[cols] = scaler.transform(X_train[cols])
X_train = X_train.to_numpy()

X_test = log_transform_feature(X_test, "redshift")
scaler = RobustScaler().fit(X_test[cols])
X_test[cols] = scaler.transform(X_test[cols])
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

The original stars dataset is also normalized for comparison in the EDA process.

In [ ]:
stars = log_transform_feature(stars, "redshift")
scaler = RobustScaler().fit(stars[columns_to_clean])
stars[columns_to_clean] = scaler.transform(stars[columns_to_clean])
stars.head()

In [ ]:
columns_to_use = ['obj_ID','alpha','delta', 'u', 'g', 'r', 'i', 'z', 'redshift', 'class']
columns_data = ['alpha','delta','u','g','r','i','z','redshift']

labels = {'GALAXY':0,'STAR':1,'QSO':2}

stars_df = stars[columns_to_use]
stars_df['class'] = stars_df['class'].map(labels)

stars_df.set_index('obj_ID')

In [ ]:
stars_df.boxplot(columns_data, by="class", figsize=(20,18))

# Exploratory Data Analysis



## Central Tendencies

To gain a better understanding of the data, we can begin with examining the central tendencies of the data. These would be the mean, median and mode.



### Means

For the means, we can take the means of each feature, pertaining to each class and compare them to one another. Putting the means together in a table, it would look like this:



In [ ]:
means_df = og_stars.groupby('class').mean()
means_df = means_df[['alpha','delta','u','g','r','i','z','redshift']].T

normalized_means_df = stars.groupby('class').mean()
normalized_means_df = normalized_means_df[['alpha','delta','u','g','r','i','z','redshift']].T

print(means_df)

print("\n")

print(normalized_means_df)

We can then graph it to a bar plot to make it easier to compare to one another. It would look like this:



In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=len(means_df), figsize=(8, 4 * len(means_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(means_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Normalized Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=len(normalized_means_df), figsize=(8, 4 * len(normalized_means_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(normalized_means_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Normalized Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

Analysis goes here, will add later



In [ ]:
mode_df = og_stars.groupby('class').apply(lambda x: x.mode().iloc[0])
mode_df = mode_df[['alpha', 'delta', 'u','g','r','i','z','redshift']].T

normalized_mode_df = stars.groupby('class').apply(lambda x: x.mode().iloc[0])
normalized_mode_df = normalized_mode_df[['alpha','delta','u','g','r','i','z','redshift']].T

print(mode_df)
print("\n")
print(normalized_mode_df)

In [ ]:
fig, axes = plt.subplots(nrows=len(mode_df), figsize=(8, 4 * len(mode_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(mode_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=len(normalized_mode_df), figsize=(8, 4 * len(normalized_mode_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(normalized_mode_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Normalized Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
median_df = og_stars.groupby('class').median()
median_df = median_df[['alpha','delta','u','g','r','i','z','redshift']].T

normalized_median_df = stars.groupby('class').median()
normalized_median_df = normalized_median_df[['alpha','delta','u','g','r','i','z','redshift']].T


print(median_df)
print("\n")
print(normalized_median_df)

In [ ]:
fig, axes = plt.subplots(nrows=len(median_df), figsize=(8, 4 * len(median_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(median_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=len(normalized_median_df), figsize=(8, 4 * len(normalized_median_df)))

# Iterate through each row and plot a bar graph
for i, (index, row) in enumerate(normalized_median_df.iterrows()):
    # Create a bar plot for each row
    ax = axes[i]
    row.plot(kind='bar', ax=ax, title=f'Normalized Row {index}')
    ax.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
stars_df.boxplot(columns_data, by="class", figsize=(20,18))

# Model Training

The following are the models that will be utizlied for training, hyperparameter tuning and testing of model

1. Logistic Regression
2. Decision Trees
3. K Nearest Neighbors \(or\)
4. Neural Network



### Logistic Regression



In [ ]:
X_train_logis = X_train.copy()
y_train_logis = y_train.copy()

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
logistic_model = SGDClassifier(loss='log_loss', eta0=0.001, max_iter=200, learning_rate='constant', random_state=1, verbose=0)

In [ ]:
max_epochs = 50

In [ ]:
from data_loader import DataLoader

In [ ]:
data_loader = DataLoader(X_train_logis, y_train_logis, batch_size=1000)

In [ ]:
from sklearn.metrics import log_loss

e = 0
is_converged = False
previous_loss = 0
labels = np.unique(y_train_logis)

# For each epoch
while e < max_epochs and is_converged is not True:
    
    loss = 0
    
    X_batch, y_batch = data_loader.get_batch()
    
    for X, y in zip(X_batch, y_batch):
        
        logistic_model.partial_fit(X, y, classes=labels)
        
        y_pred_logis = logistic_model.predict_proba(X_train_logis)
        loss += log_loss(y_train_logis, y_pred_logis)
        
    print('Epoch:', e + 1, '\tLoss:', (loss / len(X_batch)))
    
    if abs(previous_loss - loss) < 0.05:
        is_converged = True
    else:
        previous_loss = loss
        e += 1

In [ ]:
### ------END OF LOGISTIC REGRESSION------

### Decision Trees

### K Nearest Neighbors

There is no training when utilizing K Nearest Neighbors



### Neural Network 



# Hyperparameter tuning

### Logistic Regression

### Decision Trees



### K Nearest Neighbors



### Neural Network

# Model selection



# Insight and conclusion


# References